## Installs for new environments

In [1]:
import sys
!{sys.executable} -m pip install tensorflow
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pillow
!{sys.executable} -m pip install scipy

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
     |████████████████████████████████| 11.3 MB 2.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 48.8 MB/s eta 0:00:01
     |████████████████████████████████| 3.1 MB 63.3 MB/s eta 0:00:01
     |████████████████████████████████| 880 kB 62.7 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
     |████████████████████████████████| 39.3 MB 2.8 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [18]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## Imports

In [19]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import math

import datetime


## Config

In [26]:
image_dim = 256 # eventual width/height of images
input_shape = (image_dim, image_dim, 3) #(width, height, RGB)

epochs = 16
batch_size = 32
learning_rate = .001

train_dir = 'dataset/train'
validate_dir = 'dataset/validate'

save_dir = 'models/rein_val_correct'
file_name_base = 'rein_val_correct'

base_log_dir = '/logs'

## Augmentation Definition

In [27]:
train = ImageDataGenerator(    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
) # Reinforcements for training dataset

validate = ImageDataGenerator(rescale=1./255) # Reinforcements for validation dataset

## Dataset Loading

In [28]:
#loads the datasets
train_dataset = train.flow_from_directory(train_dir, target_size=(image_dim, image_dim), class_mode='categorical') 
validate_dataset = validate.flow_from_directory(validate_dir, target_size=(image_dim, image_dim), class_mode='categorical')

# tf.keras.preprocessing.image_dataset_from_directory - this can be used for no reinforcement

n_train = train_dataset.samples # number of images in training dataset
n_validate = validate_dataset.samples # number of images in validation dataset


Found 8000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.


## Dataset Loading: No Augmentation

In [21]:
train = ImageDataGenerator() # Reinforcements for training dataset

validate = ImageDataGenerator(rescale=1./255) # Reinforcements for validation dataset

#loads the datasets
train_dataset = train.flow_from_directory(train_dir, target_size=(image_dim, image_dim), class_mode='categorical') 
validate_dataset = validate.flow_from_directory(validate_dir, target_size=(image_dim, image_dim), class_mode='categorical')

# tf.keras.preprocessing.image_dataset_from_directory - this can be used for no reinforcement

n_train = train_dataset.samples # number of images in training dataset
n_validate = validate_dataset.samples # number of images in validation dataset


Found 8000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.


## CNN Design

In [29]:
class_keys = list(train_dataset.class_indices.keys())
n_classes = len(class_keys)

# defines the layers of the cnn
model = tf.keras.models.Sequential([
    Conv2D(16, (3, 3), activation='relu', padding='same', input_shape=input_shape),
    Conv2D(16, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2,2), strides=None, padding='valid'),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2,2), strides=None, padding='valid'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2,2), strides=None, padding='valid'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2,2), strides=None, padding='valid'),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(n_classes, activation='softmax')
])

## Compilation

In [30]:
log_dir = base_log_dir + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9),
              loss='MSE',
              metrics=['accuracy'])

## Evaluate and Save the initial model

In [31]:
model.evaluate(validate_dataset, batch_size=256)
model.save(save_dir + '/' + file_name_base + '_0')

63/63 [==============================] - 5s 69ms/step - loss: 0.0900 - accuracy: 0.1025
INFO:tensorflow:Assets written to: models/rein_val_correct/rein_val_correct_0/assets


## Training

In [11]:
model = tf.keras.models.load_model(save_dir + '/' + file_name_base + '_30')

In [32]:
for i in range(0,4):
    print((i+1) * epochs)
    model.fit(train_dataset,
                        steps_per_epoch=math.floor(n_train/batch_size),
                        validation_data=validate_dataset,
                        validation_steps=math.floor(n_validate/batch_size),
                        epochs=epochs,
                        callbacks=[tensorboard])
    model.save(save_dir + '/' + file_name_base + '_' + str((i+1)*epochs))


16
Epoch 1/16
250/250 [==============================] - 98s 388ms/step - loss: 0.0896 - accuracy: 0.1674 - val_loss: 0.0900 - val_accuracy: 0.2399
Epoch 2/16
250/250 [==============================] - 98s 392ms/step - loss: 0.0857 - accuracy: 0.2288 - val_loss: 0.0900 - val_accuracy: 0.2460
Epoch 3/16
250/250 [==============================] - 98s 391ms/step - loss: 0.0838 - accuracy: 0.2585 - val_loss: 0.0899 - val_accuracy: 0.2636
Epoch 4/16
250/250 [==============================] - 97s 388ms/step - loss: 0.0822 - accuracy: 0.2850 - val_loss: 0.0899 - val_accuracy: 0.2218
Epoch 5/16
250/250 [==============================] - 97s 389ms/step - loss: 0.0812 - accuracy: 0.3027 - val_loss: 0.0899 - val_accuracy: 0.2379
Epoch 6/16
250/250 [==============================] - 97s 389ms/step - loss: 0.0803 - accuracy: 0.3121 - val_loss: 0.0899 - val_accuracy: 0.2223
Epoch 7/16
250/250 [==============================] - 98s 391ms/step - loss: 0.0796 - accuracy: 0.3211 - val_loss: 0.0899 - val

250/250 [==============================] - 97s 389ms/step - loss: 0.0610 - accuracy: 0.5285 - val_loss: 0.0899 - val_accuracy: 0.1477
Epoch 10/16
250/250 [==============================] - 97s 388ms/step - loss: 0.0607 - accuracy: 0.5395 - val_loss: 0.0899 - val_accuracy: 0.1396
Epoch 11/16
250/250 [==============================] - 97s 388ms/step - loss: 0.0607 - accuracy: 0.5361 - val_loss: 0.0899 - val_accuracy: 0.1391
Epoch 12/16
250/250 [==============================] - 97s 389ms/step - loss: 0.0607 - accuracy: 0.5343 - val_loss: 0.0899 - val_accuracy: 0.1316
Epoch 13/16
219/250 [=========================>....] - ETA: 11s - loss: 0.0601 - accuracy: 0.5388

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Save Trained Model

In [19]:
model.save(save_dir + '/' + file_name_base + '_45')

INFO:tensorflow:Assets written to: models/reinforced_full/reinforced_full_45\assets
